In [1]:
import torch, math
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import v2

# Hugging Face
from transformers import AutoImageProcessor, AutoModel
from datasets import load_dataset, Image
from transformers.image_utils import load_image
from huggingface_hub import notebook_login

#notebook_login()

In [1]:
# autoreload notebook
%load_ext autoreload
%autoreload 2
import torch
# clear torch cache
torch.cuda.empty_cache()

In [ ]:
import torch
import torch.nn.functional as F  # <--- Il manquait cet import
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoImageProcessor

# Imports de tes fichiers locaux
from models.DinoV3.SemanDino import GlacierSegmenter
from models.DinoV3.GlacierDataset import GlacierDataset

# --- SETUP ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on: {device}")

# Modèle
model = GlacierSegmenter(num_classes=2)
model = model.to(device)

# Dataset
dataset = GlacierDataset(
    image_dir="dataset/clean/images/",
    mask_dir="dataset/clean/masks/",
    mode="train",
)

# DataLoader
data_loader = DataLoader(
    dataset,
    batch_size=8, 
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    drop_last=True
)

# Optimizer : On ne lui donne que les paramètres qui demandent un gradient (Le Décodeur)
# C'est une bonne pratique quand on freeze une partie du réseau.
params_to_update = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.AdamW(params_to_update, lr=1e-4)

# --- TRAINING LOOP ---

# On met le modèle en mode train UNE FOIS avant la boucle
model.train() 

pb = tqdm(data_loader, total=len(data_loader), desc="Training")

for batch in pb:
    # CORRECTION 1 : Unpacking du Tuple (pas de dictionnaire)
    images, masks = batch
    
    # Envoi sur GPU
    images = images.to(device)
    masks = masks.to(device)
        
    # Forward Pass
    outputs = model(images)
    
    # CORRECTION 2 : Gestion du 255 (Ignore Index)
    # On dit à la Loss : "Si le pixel vaut 255, ne calcule pas d'erreur dessus"
    loss = F.cross_entropy(outputs, masks, ignore_index=255)
    
    # Backward Pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Affichage
    pb.set_description(f"Loss: {loss.item():.4f}")

Training on: cuda


Loss: 0.4229:  14%|█▍        | 10/69 [08:09<46:57, 47.76s/it] 

In [7]:
from pystac_client import Client

# 1. URL de l'API STAC
STAC_URL = "https://data.geo.admin.ch/api/stac/v0.9/"

# 2. Initialisation du client
client = Client.open(STAC_URL)

# --- CORRECTIF CRITIQUE POUR L'ERREUR "DoesNotConformTo" ---
# On force le client à ignorer la validation stricte du serveur
client.add_conforms_to("ITEM_SEARCH")
# -----------------------------------------------------------

# 3. Paramètres de recherche
# Bbox: [Ouest, Sud, Est, Nord] -> Ici un rectangle autour de Lausanne
bbox_lausanne = [6.61, 46.50, 6.66, 46.54]

print("Recherche des images en cours...")

# 4. Lancement de la recherche ciblée
search = client.search(
    collections=["ch.swisstopo.lubis-luftbilder_farbe"], # La collection EXACTE qu'il vous faut
    bbox=None,
    datetime="2008-01-01/2008-12-31" # Année 2008
)

# 5. Récupération des résultats
items = list(search.items())
print(f"Nombre d'images trouvées : {len(items)}")

# 6. Affichage des liens de téléchargement
if items:
    # On prend la première image pour l'exemple
    item = items[0]
    print(f"\nImage ID: {item.id}")
    print(f"Date de prise de vue: {item.datetime}")
    
    # Recherche du fichier GeoTIFF complet (souvent sous la clé 'data' ou finissant par .tif)
    image_asset = item.assets.get("data")
    
    if image_asset:
        print(f"Lien de téléchargement (TIF) : {image_asset.href}")
        print("-> Vous pouvez télécharger ce lien pour votre CNN.")
    else:
        # Si la clé 'data' n'existe pas, on liste tout pour trouver le .tif
        print("Lien direct 'data' non trouvé, voici les assets disponibles :")
        for key, asset in item.assets.items():
            print(f"- {key} : {asset.href}")
else:
    print("Aucune image trouvée pour cette zone et cette date.")

Recherche des images en cours...
Nombre d'images trouvées : 136

Image ID: lubis-luftbilder_farbe_000-161-813
Date de prise de vue: 2008-08-18 00:00:00+00:00
Lien direct 'data' non trouvé, voici les assets disponibles :
- lubis-luftbilder_farbe_000-161-813.csv : https://data.geo.admin.ch/ch.swisstopo.lubis-luftbilder_farbe/lubis-luftbilder_farbe_000-161-813/lubis-luftbilder_farbe_000-161-813.csv
- lubis-luftbilder_farbe_000-161-813.jpg : https://data.geo.admin.ch/ch.swisstopo.lubis-luftbilder_farbe/lubis-luftbilder_farbe_000-161-813/lubis-luftbilder_farbe_000-161-813.jpg
- lubis-luftbilder_farbe_000-161-813_2056.tif : https://data.geo.admin.ch/ch.swisstopo.lubis-luftbilder_farbe/lubis-luftbilder_farbe_000-161-813/lubis-luftbilder_farbe_000-161-813_2056.tif
- lubis-luftbilder_farbe-calibration_057.pdf : https://data.geo.admin.ch/ch.swisstopo.lubis-luftbilder_farbe/lubis-luftbilder_farbe_000-161-813/lubis-luftbilder_farbe-calibration_057.pdf
